In [2]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

2024-10-28 13:52:41.443507: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-28 13:52:51.837014: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
naive_transcripts = pd.read_csv('Data/machine_learning_input_prep/all_transcripts_with_fasta_clip_m6a_paris_intra_paris_inter_postar3.bed', sep='\t')
naive_transcripts['smoops_common'] = ~naive_transcripts['control_common']

In [3]:
naive_transcripts['start'] = pd.to_numeric(naive_transcripts['start'])
naive_transcripts['end'] = pd.to_numeric(naive_transcripts['end'])

sequence_index = naive_transcripts.columns.get_loc("sequence")

def process_group(group):
    if group['strand'].iloc[0] == '+':
        sorted_group = group.sort_values(by='start')
    else:
        sorted_group = group.sort_values(by='end', ascending=False)
    
    concatenated_sequence = ''.join(sorted_group['sequence'])
    other_columns = sorted_group.iloc[:, sequence_index + 1:].apply(lambda x: ','.join(x.dropna().astype(str)), axis=0)

    first_columns = sorted_group.iloc[0][[*group.columns[3:sequence_index]]]

    return pd.Series([*first_columns.values, concatenated_sequence, *other_columns.values])

merged_naive_transcripts = naive_transcripts.groupby('transcript_id').apply(process_group).reset_index(drop=True)
merged_naive_transcripts.columns = [naive_transcripts.columns[3:]]
merged_naive_transcripts.columns = merged_naive_transcripts.columns.get_level_values(0)

/tmp/ipykernel_914597/2644908499.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_naive_transcripts = naive_transcripts.groupby('transcript_id').apply(process_group).reset_index(drop=True)


In [8]:
merged_naive_transcripts["sequence_length"] = merged_naive_transcripts["sequence"].str.len()
merged_naive_transcripts = merged_naive_transcripts[merged_naive_transcripts["sequence_length"] < 20000]
merged_naive_transcripts = merged_naive_transcripts.drop(columns=["sequence_length"])
print(merged_naive_transcripts["sequence_length"].max())

16716

In [9]:
merged_naive_transcripts.to_csv('Data/machine_learning_input_prep/all_transcripts_with_fasta_clip_m6a_paris_intra_paris_inter_postar3_collapsed.bed', sep='\t', index=False)